# Strata of exact differentials - Cross-checks

Let $X=\mathbb{P}\mathcal{H}(\mu)$ be a projectivized stratum and $\mathcal{V}\subseteq\mathcal{H}$ a globally defined sublocal system.
An exact stratum is specified using 
`ExactStratum(generalized_stratum, relative_sub_space)`
where 'generalized_stratum' determines the ambient GeneralisedStratum and `relative_sub_space` describe the linear equations among marked points, i.e., specifies which marked points lie in the same fiber.
For example 

```ExactStratum(Stratum([-5,1,1,1,0,0]), [[0,1,-1,0,0,0],[0,0,0,1,-1,0],[0,0,0,0,1,-1]])```

describes degree $4$ covers $\mathbb{P}^1\to \mathbb{P}^1$ with ramification profile $(4),(2,2),(2,1,1)$.
(Note that one could remove the two marked points with multiplicity zero and instead use 
```ExactStratum(Stratum([-5,1,1,1]), [[0,1,-1,0,0]])```, which describes the same class up to a degree shift because of the extra marked points.)

Then 

```A = ExactStratum(generalized_stratum, relative_sub_space).exact_stratum_class()```

computes the class of the locus $\operatorname{Exc}_{\lambda}$ of $\lambda$-exact differentials in the Chow ring $\operatorname{CH}^*(\mathbb{P}\Xi(\mu))$.
The result is a SymbolicExpression corresponding to a divisorial tautological class as a product in $\xi$ and boundary divisors $D_{\Gamma}$.
Afterwards `A.to_ELG(symbolic_class)` converts the class to an ELGTautClass, i.e. a class in the tautological ring of the moduli space of multi-scale differentials. 
The pushforward $\rho_*[\overline{\operatorname{Exc}}_{\lambda}]$ in the Chow ring of $\overline{\mathcal{M}}_{g,n}$ under the forgetful map $\rho:\mathbb{P}\Xi(\mu)\to\overline{\mathcal{M}}_{g,n}$
is then obtained by applying `A.pushforward(elg_taut_class)`.




In [1]:
from admcycles import *
from admcycles.diffstrata import *

from exactstrata import *

In [2]:
import_statements(PowerSeriesRing)

from sage.rings.power_series_ring import PowerSeriesRing


## First cross-check: Elliptic curves with marked 2-torsion
Consider the closure of the locus 
\begin{equation}
Z=\{ (E,p,q,r,s)\in\mathcal{M}_{1,4}\,:\, \exists \text{ degree $2$ covering} f:E\to\mathbb{P}^1 \text{ ramified at } p,q,r,s\}
\end{equation} inside $\overline{\mathcal{M}}_{1,4}$.
We can compute the class of the closure of the stratum of exact differentials $\operatorname{Exc}(-3,1^4)$ in $\mathcal{H}(-3,1^4)$ and push it forward to $\mathcal{M}_{1,4}$.

In [3]:
A= ExactStratum(Stratum([-3,1,1,1]),[])
divtaut =  A.exact_stratum_class()
print("Divisorial tautological class: ", divtaut)
cl= A.pushforward(A.to_ELG(divtaut))
show("Simplify pushforward using Faber-Zagier relations:")
print(cl.FZsimplify())
cl.basis_vector()

Avoiding blowup
Divisorial tautological class:  (D_12 + D_13 + D_14 + xi)^2 + (D_12 + xi)*D_4 + (D_13 + xi)*D_5 + (D_14 + xi)*D_6 + 2*D_11*xi


'Simplify pushforward using Faber-Zagier relations:'

Graph :      [1] [[1, 2, 3, 4]] []
Polynomial : 75/2*(kappa_3)_0 - 45/4*(kappa_1*kappa_2)_0 + 3/4*(kappa_1^3)_0 + 3/2*(kappa_2)_0*psi_1 + 3/2*(kappa_2)_0*psi_2 + 3/2*(kappa_2)_0*psi_3 + 3/2*(kappa_2)_0*psi_4


(75/2, -45/4, 3/4, 3/2, 0, 3/2, 0, 3/2, 0, 3/2, 0, 0)

Avoiding blowup means that we use the exact formulas in g=0,1 instead of the recursive blow up procedure.
We can also use the blowup recursion instead.

In [4]:
(A.exact_stratum_class() - A.exact_stratum_class(avoiding_blowup = False)).expand()

Avoiding blowup


0

To cross-check our computation we compute the class in $\overline{\mathcal{M}}_{1,4}$ using `admcycles` instead.


In [5]:
print('Admcycles computation: ', Hyperell(1,4,0).basis_vector())
(Hyperell(1,4,0)-cl).is_zero()

Admcycles computation:  (75/2, -45/4, 3/4, 3/2, 0, 3/2, 0, 3/2, 0, 3/2, 0, 0)


True

## Sanity checks

A holomorphic stratum or with only simple poles does not have exact differentials, thus the fundamental class has to be zero in this case.
ELGTautClass cannot be simplified to see if they are zero or not, so instead we pushforward to $\overline{\mathcal{M}}_{g,n}$ and verify that the pushforward is zero.
Here we use that the Chow class of a subvariety in a projective variety is zero if and only if the subvariety is empty.

In [6]:
# Holomorphic stratum
A= ExactStratum(Stratum([2]),[])
cl = A.exact_stratum_class()
show(cl)
A.to_ELG(cl) == A.X.ZERO

4*D_1*xi^3 + xi^4

True

Similarly exact differentials cannot have simple poles (since then the residue gives a non-zero period).

In [7]:
# Stratum with simple zeros
A= ExactStratum(Stratum([2,-1,-1]),[])
cl = A.exact_stratum_class(avoiding_blowup = True)
print(cl)
show(cl==A.X.ZERO) # Note that this is only a sufficient but not necessary check
# since we do not know all relation among ELGTautClasses 

Tautological class on Stratum: (2, -1, -1)
with residue conditions: []




True

In the following example with simple poles the resulting tautological class is highly complicated. Nonetheless it reduces to zero.

In [8]:
# Stratum with simple zeros
A= ExactStratum(Stratum([-2,-1,2,1]),[])
cl = A.exact_stratum_class(avoiding_blowup = True)
print(cl)
A.pushforward(A.to_ELG(cl)).basis_vector()

Avoiding blowup
-((D_11 + D_12 + 2*D_13 + xi)^2 + (D_11 + D_12 + 2*D_13 + xi)*D_3 + (D_12 + xi)*D_7 + 2*(2*D_13 + xi)*D_9 + D_6*xi)*(D_0 + D_1 + D_12 + 2*D_2 + D_4 + 3*D_5 + D_6 + D_7 + xi)


(0)

Here is an expression of the tautological class in terms of decorated strata classes.

In [9]:
A.pushforward(A.to_ELG(cl)).simplify()

Graph :      [1] [[1, 2, 3, 4]] []
Polynomial : -1/2*(kappa_1)_0*psi_1^3 + 1/2*psi_1^4 + 9/2*psi_1^3*psi_3 + 2*psi_1^3*psi_4

Graph :      [0] [[6, 7, 1, 2, 3, 4]] [(6, 7)]
Polynomial : -1/24*psi_1^3

Graph :      [0, 1] [[1, 2, 3, 4, 6], [7]] [(6, 7)]
Polynomial : -psi_1*psi_6*psi_7

Graph :      [0, 1] [[1, 2, 4, 6], [3, 7]] [(6, 7)]
Polynomial : 1/2*(kappa_1)_1*psi_1*psi_7 - 9/2*psi_1*psi_3*psi_7 - 1/2*psi_1*psi_7^2

Graph :      [0, 1] [[2, 3, 6], [1, 4, 7]] [(6, 7)]
Polynomial : (kappa_1)_1*psi_1^2 - 3*psi_1^3 - 4*psi_1^2*psi_4 - 4*psi_1^2*psi_7

Graph :      [0, 1] [[2, 4, 6], [1, 3, 7]] [(6, 7)]
Polynomial : 1/2*(kappa_1)_1*psi_1^2 - psi_1^3 - 9/2*psi_1^2*psi_3 - 1/2*psi_1^2*psi_7

Graph :      [0, 1] [[3, 4, 6], [1, 2, 7]] [(6, 7)]
Polynomial : -8*psi_1^3

Graph :      [0, 0] [[1, 2, 4, 9], [7, 8, 3, 10]] [(7, 8), (9, 10)]
Polynomial : 1/24*psi_1*psi_10

Graph :      [0, 0] [[2, 3, 9], [7, 8, 1, 4, 10]] [(7, 8), (9, 10)]
Polynomial : 1/12*psi_1^2

Graph :      [0, 0] [[2, 4, 9]

## Cross-check: Double ramification in genus 1
Fix a collection of non-negative integers $\underline{a}=(a_1,\ldots,a_n)$ with $\sum_i a_i=0$. We can define the following divisor class in $\overline{\mathcal{M}}_{1,n}$: Define $D_{\underline{a}}$ to be the closure of the locus where $\mathcal{O}(\sum_i a_ip_i)\simeq \mathcal{O}$. We let $N=\{1,\ldots,n\}$.
Then then divisor class is given by $D_{\underline{a}}= \left(-1+\dfrac{1}{2}\sum_i a_i^2\right)\left(\lambda+\sum_{S\supseteq N} \delta_{0;S}\right)- \sum_{S\nsupseteq N} \sum_{{i,j}\subseteq S} a_ia_j\delta_{0;S}$.

We restrict to the special case $n=2,\underline{a}=(a,-a)$.

In [10]:
def torsion_class(a):
    return  (-1+a^2)*(lambdaclass(1,1,2)+sepbdiv(0,[1,2],1,2))

We can also use exact differential to compute the class as follows. If $\mathcal{O}(ap+aq)\simeq\mathcal{O}$ on an elliptic curve $E$, there exists a degree $a$ covering to $\mathbb{P}^1$ with zeros and poles only at $p$ and $q$. Generically the remaining ramification is simple.
We can thus compute the class of $D_{\underline{a}}$ by considering the locus of exact differentials in the stratum $\mathcal{H}(-a-1,a-1,1,1)$ and then forgetting the two simple zeros. The degree of the pushforward map is $2$.

In [11]:
def torsion_exact(a):
    X= Stratum([-a-1,a-1,1,1])
    A= ExactStratum(X, [])
    taut = A.exact_stratum_class()
    
    return A.pushforward(A.to_ELG(taut)).forgetful_pushforward([3,4])/2


In [12]:
for a in [3..8]:
    print((torsion_class(a)-torsion_exact(a)).is_zero())
    

Avoiding blowup
True
Avoiding blowup
True
Avoiding blowup
True
Avoiding blowup
True
Avoiding blowup
True
Avoiding blowup
True


## Non-existence of ramified coverings
It is in general a hard problem to decide whether there exists a ramified covering $f:\mathbb{P}^1\to\mathbb{P}^1$ with a given ramification profile. One of the necessary conditions is given by Riemann-Hurwitz but this codition is not sufficient. The easiest counter example is in degree $4$ with ramification $(3,1),(2,2),(2,2)$. Every other ramification profile in degree $4$ is realizable.
We can also compute the class of the corrresponding stratum of $\lambda$-exact differentials.
Then the class is non-zero if and only if there exists a covering with the specified ramification profile.

In [13]:
A= ExactStratum(Stratum([-4,-2,1,1,1,1]),[[0,0,1,-1,0,0],[0,0,0,0,1,-1]])
cl = A.exact_stratum_class()
print(cl)
A.pushforward(A.to_ELG(cl)).is_zero()

-(D_0 + D_1 + 2*D_12 + 2*D_13 + 2*D_14 + 2*D_15 + D_2 + 3*D_20 + 3*D_21 + 3*D_22 + 3*D_23 + 3*D_24 + 3*D_25 + D_3 + D_4 + D_5 + xi)*(D_0 + 2*D_12 + 2*D_13 + 4*D_16 + 4*D_17 + 3*D_20 + 3*D_21 + 3*D_22 + 3*D_23 + 3*D_24 + 3*D_25 + 5*D_26 + 3*D_27 + 3*D_28 + 3*D_29 + 3*D_6 + xi)*(3*D_11 + 2*D_14 + 2*D_15 + 4*D_18 + 4*D_19 + 3*D_20 + 3*D_25 + 5*D_26 + 3*D_27 + D_5 + xi)


True

Here is an expansion of the class in the tautological ring of $\overline{\mathcal{M}}_{0,6}$.

In [14]:
cl.expand().simplify()

-3*D_0^2*D_11 - 3*D_0*D_1*D_11 - 12*D_0*D_11*D_12 - 6*D_1*D_11*D_12 - 12*D_11*D_12^2 - 12*D_0*D_11*D_13 - 6*D_1*D_11*D_13 - 24*D_11*D_12*D_13 - 12*D_11*D_13^2 - 2*D_0^2*D_14 - 2*D_0*D_1*D_14 - 6*D_0*D_11*D_14 - 8*D_0*D_12*D_14 - 4*D_1*D_12*D_14 - 12*D_11*D_12*D_14 - 8*D_12^2*D_14 - 8*D_0*D_13*D_14 - 4*D_1*D_13*D_14 - 12*D_11*D_13*D_14 - 16*D_12*D_13*D_14 - 8*D_13^2*D_14 - 4*D_0*D_14^2 - 8*D_12*D_14^2 - 8*D_13*D_14^2 - 2*D_0^2*D_15 - 2*D_0*D_1*D_15 - 6*D_0*D_11*D_15 - 8*D_0*D_12*D_15 - 4*D_1*D_12*D_15 - 12*D_11*D_12*D_15 - 8*D_12^2*D_15 - 8*D_0*D_13*D_15 - 4*D_1*D_13*D_15 - 12*D_11*D_13*D_15 - 16*D_12*D_13*D_15 - 8*D_13^2*D_15 - 8*D_0*D_14*D_15 - 16*D_12*D_14*D_15 - 16*D_13*D_14*D_15 - 4*D_0*D_15^2 - 8*D_12*D_15^2 - 8*D_13*D_15^2 - 12*D_0*D_11*D_16 - 12*D_1*D_11*D_16 - 24*D_11*D_12*D_16 - 24*D_11*D_13*D_16 - 8*D_0*D_14*D_16 - 8*D_1*D_14*D_16 - 24*D_11*D_14*D_16 - 16*D_12*D_14*D_16 - 16*D_13*D_14*D_16 - 16*D_14^2*D_16 - 8*D_0*D_15*D_16 - 8*D_1*D_15*D_16 - 24*D_11*D_15*D_16 - 16*D_12*D_15

The ramification profile  $(4),(2,2),(2,1,1)$ on the other hand is realizable and therefore the class in in the tautological ring has to be non-zero.

In [15]:
A= ExactStratum(Stratum([-5,1,1,1]),[[0,1,-1,0]])
cl = A.exact_stratum_class()
show(cl)
show(A.to_ELG(cl)==A.X.ZERO)
A.pushforward(A.to_ELG(cl)).is_zero()

-3*D_2 - xi

False

False

The ramification profile $(2,2), (3,2), (2,1,1)$ is not admissable, since the degree is not correct.

In [16]:
A= ExactStratum(Stratum([-3,-3,2,1,1]),[[0,0,0,1,-1], [0,0,1,-1,0]])
cl = A.exact_stratum_class()
print(cl)
show(A.to_ELG(cl)==A.X.ZERO)
A.pushforward(A.to_ELG(cl)).is_zero()

-(D_0 + D_1 + D_2 + D_3 + D_4 + D_5 + xi)*(D_2 + D_4 + 4*D_6 + 5*D_8 + xi)*(D_3 + D_5 + 4*D_7 + 5*D_8 + 3*D_9 + xi)


True

True

Any ramification profile with a totally ramified point is realizable, for example $(4), (2,2), (2,1,1)$.

In [17]:
A= ExactStratum(Stratum([-5,1,1,1]),[[0,1,-1,0]])
cl = A.exact_stratum_class()
print(cl)
show(A.to_ELG(cl)==A.X.ZERO)
A.pushforward(A.to_ELG(cl)).is_zero()

-3*D_2 - xi


False

False

Sometimes the class of $\lambda$-exact differentials inside $\overline{\mathcal{M}}_{g,n}$ can be computed in several ways. For example for the ramification profile $(4),(2,2),(2,1,1)$ we can either mark all $6$ points or not the simple marked zeros.
In the following we compute the class in both ways.

In [18]:
A = ExactStratum(Stratum([-5,1,1,1,0,0]), [[0,1,-1,0,0,0],[0,0,0,1,-1,0],[0,0,0,0,1,-1]])
cl = A.exact_stratum_class()
print(cl)

A.pushforward(A.to_ELG(cl)).basis_vector()

-(3*D_10 + 3*D_11 + 4*D_16 + 6*D_18 + 6*D_19 + 6*D_20 + 2*D_24 + 3*D_27 + 3*D_28 + 2*D_33 + 2*D_35 + 6*D_39 + 2*D_4 + 6*D_40 + 6*D_41 + 6*D_43 + 6*D_44 + 6*D_46 + 6*D_47 + 6*D_48 + xi)*(3*D_12 + 3*D_15 + 4*D_16 + 4*D_17 + 3*D_2 + 6*D_20 + 6*D_23 + 3*D_29 + 4*D_3 + 3*D_32 + 6*D_41 + 4*D_42 + 6*D_45 + 6*D_48 + xi)*(3*D_13 + 3*D_14 + 4*D_17 + 6*D_21 + 6*D_22 + 6*D_23 + 2*D_24 + 2*D_25 + 2*D_26 + 3*D_27 + 3*D_28 + 3*D_29 + 3*D_30 + 3*D_31 + 3*D_32 + 2*D_34 + 2*D_36 + 2*D_37 + 2*D_38 + 6*D_39 + 6*D_40 + 6*D_41 + 4*D_42 + 6*D_43 + 6*D_44 + 6*D_45 + 6*D_46 + 6*D_47 + D_49 + 2*D_7 + xi)


(2)

In [19]:
A = ExactStratum(Stratum([-5,1,1,1]), [[0,1,-1,0]])
cl = A.exact_stratum_class()
print(cl)

A.pushforward(A.to_ELG(cl)).basis_vector()

-3*D_2 - xi


(1)

Note that if that the forgetful map from $\overline{\mathcal{M}}_{0,6} \to \overline{\mathcal{M}}_{0,4}$ has degree $2$ on the locus of exact differentials and thus the two results agree.

  # Divisors from Hurwitz spaces (after Mullane)  
  Let $f = (f_1,...,f_n)$ for $f_i \in\mathbb{Z}$ and $\sum_{i=1}^n f_i =0$. 
  Let $D_f$  be the closure of the locus of pointed rational curves $(X,p_1,\ldots,p_n)\in\mathcal{M}_{0,n}$ such that there exists a map $f : X \to \mathbb{P}^1$  with  ramification order $f_i$ at $p_i$  iff $i >0$ and one fibre consisting of the $p_i$ such that $f_i <0$, where the ramification at such $p_i$ is $|f_i| − 1$.
Mullane showed that $$ D_f= \sum_{i=1}^n f(\{i\})\psi_i −\sum_{S:1\in S} f(S)\delta_{0:S},$$ where
   $S\subseteq \{1,\ldots,n\}, |S|\geq 2, |S^c|\geq 2$ and $f(S) = \dfrac{1}{2} |\sum_{i\in S} f_i|$.
   

In [20]:
def hurwitz_divisor(f):
    n = len(f)
    pts = Set(range(1,n+1))
    pts_psi = sum( 1/2*abs(f[i-1])*psiclass(i,0,n ) for i in [1..n])
    S = Subsets(pts)
    set_psi = 0
    for subset in S:
        if 1 in subset and len(subset)>=2 and len(pts-subset)>=2:
            
            f_subset = 1/2 * abs(sum(f[i-1] for i in subset))
            set_psi += f_subset * sepbdiv(0,subset,0,n)
    return pts_psi-set_psi


# Converts a tuple to a signature of a meromoprhic stratum
# Potentially adds simple zeros (of the differential)
def f_to_stratum(f):
    g = f
    d = -sum(fi for fi in f if fi < 0)
    #Riemann-Hurwitz 
    R = 2*d-2
    
    for i, fi  in enumerate(g):
        if fi < 0:
            g[i] = fi-1
    
    assert sum(g) <=-2
    # Add simple zeros so that Riemann-Hurwitz is satisfied
    g = g + [1 for i in range(0,-2-sum(g))]
    
    return g


# Computes the class of the exact stratum corresponding to the partition f
def hurwitz_exact(f):
    
    sig = f_to_stratum(f)
    
    #The degree of the forgetful map
    deg =len(sig)-len(f)
    
    A = ExactStratum(Stratum(sig),[])
    
    taut_class = A.pushforward(A.to_ELG(A.exact_stratum_class()))
    
    # If we added addtionaly simple zeros we now need to forget them
    if deg > 0:
        forget_points = [1..len(sig)][-deg:]
        
        return taut_class.forgetful_pushforward(forget_points)
    else:
        return taut_class
    
    
                     
    

        
        


In [21]:
hurwitz_divisor([-2,-1,1,1,1]).basis_vector()

(0, 0, 1, 0, 0)

In [24]:
hurwitz_exact([-2,-1,1,1,1]).basis_vector()


(0, 0, 1, 0, 0)

In [22]:
hurwitz_divisor([-2,-2,-1,5]).basis_vector()

(0)

In [23]:
hurwitz_exact([-2,-2,-1,5]).basis_vector()

(0)